In [105]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import pandas as pd
from sklearn.externals import joblib

In [5]:
scaled_data = pd.read_csv('scaled_data_2017-04-16.csv')

In [94]:
combined_data = pd.read_csv('combined_data_2017-04-16.csv')

In [47]:
X_train, X_test, y_train, y_test = train_test_split(scaled_data[['GyrX','GyrY','GyrZ','AccX','AccY','AccZ']], scaled_data['type'], test_size=0.5, random_state=42)

In [95]:
X_train, X_test, y_train, y_test = train_test_split(combined_data[['GyrX','GyrY','GyrZ','AccX','AccY','AccZ']], combined_data['type'], test_size=0.5, random_state=42)

In [96]:
print scaled_data['type'].value_counts()/scaled_data['type'].count()
print y_train.value_counts()/y_train.count()
print y_test.value_counts()/y_test.count()

noshot    0.327961
FHG       0.224191
BHG       0.192620
FHV       0.160384
BHV       0.094845
Name: type, dtype: float64
noshot    0.331202
FHG       0.221689
BHG       0.189981
FHV       0.160938
BHV       0.096190
Name: type, dtype: float64
noshot    0.324720
FHG       0.226692
BHG       0.195258
FHV       0.159830
BHV       0.093500
Name: type, dtype: float64


In [97]:
clf = RandomForestClassifier(n_estimators=20)
clf = clf.fit(X_train, y_train)

In [98]:
pd.crosstab(y_train, clf.predict(X_train))

col_0,BHG,BHV,FHG,FHV,noshot
type,,,,,
BHG,713,0,0,0,0
BHV,1,358,0,1,1
FHG,0,0,832,0,0
FHV,0,0,0,604,0
noshot,0,0,0,0,1243


In [99]:
pd.crosstab(y_test, clf.predict(X_test))

col_0,BHG,BHV,FHG,FHV,noshot
type,,,,,
BHG,662,19,20,4,28
BHV,34,277,5,11,24
FHG,18,2,778,17,36
FHV,14,10,30,519,27
noshot,27,12,35,32,1113


In [100]:
prob_analysis = pd.DataFrame(clf.predict_proba(X_test),X_test.index, columns=['BHG','BHV','FHG','FHV','noshot'])
prob_analysis['actual'] = y_test
prob_analysis['pred'] = clf.predict(X_test)
prob_analysis['max_prob'] = prob_analysis.apply(lambda x: max(x[['BHG','BHV','FHG','FHV','noshot']]), axis=1)

In [101]:
prob_analysis[prob_analysis['actual'] != prob_analysis['pred']]

,BHG,BHV,FHG,FHV,noshot,actual,pred,max_prob
3007,0.40,0.60,0.00,0.00,0.00,BHG,BHV,0.60
3816,0.05,0.00,0.00,0.00,0.95,BHG,noshot,0.95
3025,0.35,0.05,0.40,0.10,0.10,BHG,FHG,0.40
6516,0.10,0.35,0.00,0.00,0.55,BHV,noshot,0.55
6693,0.05,0.40,0.00,0.05,0.50,BHV,noshot,0.50
3076,0.35,0.05,0.15,0.40,0.05,BHG,FHV,0.40
1061,0.55,0.00,0.15,0.15,0.15,FHG,BHG,0.55
6182,0.05,0.10,0.40,0.40,0.05,FHV,FHG,0.40
5926,0.00,0.00,0.15,0.50,0.35,noshot,FHV,0.50
5440,0.00,0.00,0.65,0.30,0.05,FHV,FHG,0.65


In [102]:
prob_analysis[prob_analysis['actual'] != prob_analysis['pred']].to_csv('probs.csv')

In [106]:
joblib.dump(clf, 'prod_model.pkl') 

['prod_model.pkl']